# 02: Advanced Functions

- Functions as variables
- Higher order Functions
- Closure
- Decorators

## Functions as variables

Most likely, you have only seen functions as something you declare and call, separate from variables you can modify, set, or operate with. However, in languages like Python, a function can be treated just like any other variable (as an object).

In [61]:
# declare an empty class, just to see what a
# "normal" object will be like
class Example(object):
    pass

# print the object and its type
print(Example(), type(Example()))

# declare some functions 
def multiply_by_2(x):
    return x*2

def divide_by_2(x):
    return x/2

print(multiply_by_2, type(multiply_by_2))
print(divide_by_2, type(divide_by_2))

<__main__.Example object at 0x000001F38163DFA0> <class '__main__.Example'>
<function multiply_by_2 at 0x000001F381C6F940> <class 'function'>
<function divide_by_2 at 0x000001F3820B5EE0> <class 'function'>


Since a function is just a variable, it can be assigned to just like any other variable. Then, you can call it just like it was a normal function.

In [62]:
fun = multiply_by_2

print(fun(1))

fun = divide_by_2

print(fun(4))

2
2.0


Variables can be passed into and from functions. If functions are just like any other variable, then they can be passed into and from functions. A function which takes or returns functions is called a higher order function.

In [63]:
def apply_operation(operation, x):
    # take a function as a parameter as if it was any
    # other parameter, and also call it just like the
    # above example
    return operation(x)

print(apply_operation(multiply_by_2, 5))
print(apply_operation(divide_by_2, 8))

def get_some_function():
    # since we can treat a function as any other variable,
    # we can return it like any other value, just like
    # saying "return 5"
    return multiply_by_2

print(apply_operation(get_some_function(), 10))


10
4.0
20


This works with functions that you don't declare too. Any function.

In [64]:
import math

print(apply_operation(math.sqrt, 16))
print(apply_operation(math.log2, 8))

4.0
3.0


In [65]:
class ExampleTwo(object):
    def __init__(self, n):
        self.n = n;

    def print_hi(self):
        for _ in range(self.n):
            print("hi")

ex = ExampleTwo(5)
fun = ex.print_hi

print("running fun:")
fun()

running fun:
hi
hi
hi
hi
hi


It works with methods too, and you can see that the instance variable `n` was still accessible.

## Closure

Hopefully the idea of functions as any other object variable has stuck. Now we'll see what kind of data is stored in each function object. Take a careful look at this example.

In [66]:
def get_distance_fn(x1, y1):
    point_1 = (x1, y1)

    def inner(x2, y2):
        return math.dist(point_1, (x2, y2))

    return inner

distance_func = get_distance_fn(5, 4)

print(distance_func(7, 4))
print(distance_func(5, -4))

2.0
8.0


We saw how `self` was preserved and we could use `self.n` in a previous example, however this appears pretty different. See what happens when calling `inner`

In [67]:
try:
    inner(5, 5)
except NameError as e:
    print("error:", e)

error: name 'inner' is not defined


This should make some sort of sense. The declaration of `inner` was indented, which means it is part of the `get_distance_fn` function. If we were to use any other variable you should be able to know that it is clearly not accessible:

In [68]:
def dummy_fn():
    not_accessible_outside = 5

try:
    print(not_accessible_outside)
except NameError as e:
    print("error:", e)

error: name 'not_accessible_outside' is not defined


Remember how variable scope works:

- Variables declared in an inner block are not accessible in a outside blocks
- Variables declared in outside blocks are accessible from an inner block

The reason why `not_accessible_outside` is not accessible is because it is a local variable declared within the scope of `dummy_fn` and so it is not accessible from the global scope. Likewise, the `inner` function was declared within the scope of `get_distance_fn` and, since functions are just variables, is just a local variable to `get_distance_fn`.

However, note the second point. It's the reason why this is possible:

In [69]:
global_variable = 5

def dummy_fn_2():
    print(global_variable*2)

dummy_fn_2()

10


Since `dummy_fn_2` is declared in the global scope with `global_variable`, the scope of `dummy_fn_2` is inside the global scope, and so `global_variable` is accessible from inside `dummy_fn_2`. However, back to the first example:

```python
def get_distance_fn(x1, y1):
    point_1 = (x1, y1)

    def inner(x2, y2):
        return math.dist(point_1, (x2, y2))

    return inner
```

If you imagine the inside of `get_distance_fn` was the global scope, and `x1` and `y1` were global variables, `inner` is just a function declared within that scope. Just like how `global_variable` was accessible within `dummy_fn_2`, `x1` and `y1` are accessible from within `inner`. So, it makes perfect sense according to our scope rules that we can access `x1`, `y1`, and `point_1`.

The only strange part is that we are returning `inner`, and it is still able to use `point_1` in later function calls, even though the original `point_1` is no longer accessible after we returned from `get_distance_fn`.

However, remember that functions are just objects. It turns out that the `inner` function stores in itself the variables that were accessible to it when it was declared.

In [70]:
print(distance_func.__closure__)

(<cell at 0x000001F38163DFD0: tuple object at 0x000001F381942200>,)


Where are the int values? Well, since `inner` doesn't use them, why would Python bother storing them? Here are a few more examples.

In [71]:
def create_quadratic_fn(a, b, c):
    def inner(x):
        return a*x*x + b*x + c
    # this will modify the c in inner, even
    # though the assignment happened after
    c = 100
    return inner

fun = create_quadratic_fn(1, 2, 3)
print(fun(1))
print(fun.__closure__)

103
(<cell at 0x000001F381876B50: int object at 0x00007FFFEC4516A0>, <cell at 0x000001F3818765B0: int object at 0x00007FFFEC4516C0>, <cell at 0x000001F3818761C0: int object at 0x00007FFFEC452300>)


In [72]:
import random

def function_repeater(fun, n):
    def inner():
        for _ in range(n):
            print(fun())
    return inner

def function_to_repeat():
    return random.randint(100, 200)

repeated = function_repeater(function_to_repeat, 5)
repeated()
print(repeated.__closure__)

192
197
118
182
102
(<cell at 0x000001F38163D520: function object at 0x000001F3820B5CA0>, <cell at 0x000001F3820C3FD0: int object at 0x00007FFFEC451720>)


This example combines both closure and functions as parameters. To summarize:
- Functions can be declared in places other than the global scope
- Functions can store and use the variables accessible to them in the declaring scope, even if those variables later become inaccessible otherwise

## Anonymous Functions

Notice that `function_to_repeat` is declared in the global scope, just to be used as a parameter. If we don't ever want to use `function_to_repeat`, we can use an anonymous function to give a function to `function_repeater` without adding a function to the global scope:

In [73]:
repeated = function_repeater(lambda: random.randint(100, 200), 5)
repeated()

161
142
146
189
131


The syntax for an anonymous function in Python is to write `lambda`, a parameter list, then a colon, and a single expression which is the return value. Another example:

`lambda x, y: x*y`

The restriction that you can only use a single expression is just a Python thing. In the future, we'll see JavaScript anonymous functions that are basically just any other function.

A very common usage of anonymous functions is when using functions like `map` in Python.

In [74]:
strings = [
    "    hELlo thEre ",
    "hoWs it Going ",
    " ok BYE now"
]

print("".join(
    map(lambda x: x.lower().strip()+",\n", strings)
))

hello there,
hows it going,
ok bye now,



The map function takes an iterable (such as a list) and returns a new iterable with the function given applied to each element.

`map(fn, [x1, x2, x3...]) -> [fn(x1), fn(x2), fn(x3)...]`

In this case, we're taking a list of strings and making each string lowercase, removing leading and trailing whitespace, and adding `",\n"` to the end of each string.

Since this function that performs this specific string operation is not very useful to us other than for this `map` call, we can use an anonymous function to avoid declaring a new function.

## `*args` and `**kwargs`

## Decorators